<a href="https://colab.research.google.com/github/GailB-ca/digital-display-reader/blob/main/Digit_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import pickle
import numpy as np
from google.colab import drive


In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
with tf.compat.v1.Session(config=config):
    pass  # init sessin with allow_growth


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filename = "/content/drive/MyDrive/Colab Notebooks/training_df.pk"
training_numbers_df = pickle.load(open(filename, 'rb'))

In [ ]:
training_numbers_df.head()

,box_order,class,image_id,image_path,xmax,xmin,ymax,ymin,width,height,portrait,img_num_arr,img_num_width,img_num_height,img_num_arr_resize,img_num_resize_height,class_arr,img_num_arr_resize_3d
0,0.0,1,0.0,/Users/gbishop/lighthouse/data_bootcamp/digita...,2166,1740,2070,1799,426,271,0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",271,426,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",628,[1],"[[[255], [255], [255], [255], [255], [255], [2..."
1,1.0,5,0.0,/Users/gbishop/lighthouse/data_bootcamp/digita...,2186,1710,1775,1514,476,261,0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",261,476,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",729,[5],"[[[255], [255], [255], [255], [255], [255], [2..."
2,2.0,0,0.0,/Users/gbishop/lighthouse/data_bootcamp/digita...,2131,1690,1535,1194,441,341,0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",341,441,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",517,[0],"[[[255], [255], [255], [255], [255], [255], [2..."
3,3.0,9,0.0,/Users/gbishop/lighthouse/data_bootcamp/digita...,2661,2165,1785,1469,496,316,0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",316,496,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",627,[9],"[[[255], [255], [255], [255], [255], [255], [2..."
4,4.0,4,0.0,/Users/gbishop/lighthouse/data_bootcamp/digita...,2601,2135,1455,1154,466,301,0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",301,466,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",619,[4],"[[[255], [255], [255], [255], [255], [255], [2..."


In [ ]:
#baseModel = ResNet50(weights='imagenet', include_top=False,
baseModel = ResNet50(weights=None, include_top=True,
                    input_tensor=Input(shape=(600, 400, 3)),
                    classes=10)

In [ ]:
# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=25,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=False,
    fill_mode="nearest",
    validation_split=0.2)

testAug = ImageDataGenerator()

In [ ]:
X = np.array(training_numbers_df['img_num_arr_resize'].values.tolist())
Y = np.array(training_numbers_df['class_arr'].values.tolist())

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train = tf.keras.applications.resnet50.preprocess_input(X_train)
Y_train = tf.keras.utils.to_categorical(Y_train)

In [ ]:
learning_rate = 1e-4
num_epochs = 5
batch_size=16

In [ ]:
trainGen = trainAug.flow(X_train, Y_train, batch_size=batch_size, subset="training")
valGen = trainAug.flow(X_train, Y_train, batch_size=batch_size, subset="validation")
testGen = testAug.flow(X_test, Y_test, batch_size=batch_size)

In [ ]:
print("Size of Training Data: ", trainGen.n)

Size of Training Data:  695


In [ ]:
opt = Adam(lr=learning_rate, 
           decay=learning_rate / num_epochs)
baseModel.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
print("Steps per epoch", trainGen.n // batch_size)
print("Number of epochs", num_epochs)
print("Validation data", valGen.n)

Steps per epoch 43
Number of epochs 5
Validation data 173


In [ ]:
baseModel.fit_generator(trainGen,
                   steps_per_epoch=trainGen.n // batch_size, 
                   epochs=num_epochs,
                   validation_data=valGen,
                   validation_steps=valGen.n // batch_size,
                   verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
43/43 [==============================] - 44s 1s/step - loss: 0.0800 - accuracy: 0.9013 - val_loss: 0.1991 - val_accuracy: 0.6687
Epoch 2/5
43/43 [==============================] - 44s 1s/step - loss: 0.0679 - accuracy: 0.9249 - val_loss: 0.1566 - val_accuracy: 0.7625
Epoch 3/5
43/43 [==============================] - 43s 993ms/step - loss: 0.0681 - accuracy: 0.9146 - val_loss: 0.1252 - val_accuracy: 0.8250
Epoch 4/5
43/43 [==============================] - 44s 1s/step - loss: 0.0629 - accuracy: 0.9219 - val_loss: 0.3604 - val_accuracy: 0.6187
Epoch 5/5
43/43 [==============================] - 44s 1s/step - loss: 0.0515 - accuracy: 0.9558 - val_loss: 0.0844 - val_accuracy: 0.8875
